In [1]:
# Setup directories
from pathlib import Path
basedir = Path().absolute()
libdir = basedir.parent.parent.parent

# Other imports
import pandas as pd
import numpy as np
from datetime import datetime

from ioos_qc.plotting import bokeh_multi_plot

from bokeh import plotting
from bokeh.layouts import gridplot
plotting.output_notebook()

ModuleNotFoundError: No module named 'ioos_qc.plotting'

In [ ]:
# Install QC library
#!pip install git+git://github.com/ioos/ioos_qc.git

# # Alternative installation (install specific branch):
!pip uninstall -y ioos_qc
!pip install git+git://github.com/ioos/ioos_qc.git@new_configs

# # Alternative installation (run with local updates):
# !pip uninstall -y ioos_qc
# import sys
# sys.path.append(str(libdir))

In [ ]:
from ioos_qc.config import Config
# QC configuration
# For sea water temperature in degrees C
# This configuration is used to call the corresponding method in the ioos_qc library
# See documentation for description of each test and its inputs: 
#   https://ioos.github.io/ioos_qc/api/ioos_qc.html#module-ioos_qc.qartod

qc_config = {
    'sea_surface_height_above_sea_level': {
        'qartod': {
          "gross_range_test": {
            "fail_span": [-10,10],
            "suspect_span": [-2,3]
          },
          "flat_line_test": {
            "tolerance": 0.001,
            "suspect_threshold": 10800,
            "fail_threshold": 21600
          },
          "rate_of_change_test": {
            "threshold": 0.001
          },
          "spike_test": {
            "suspect_threshold": 0.8,
            "fail_threshold": 3
          },
          "aggregate": {}
        }
    }
}
c = Config(qc_config)

In [ ]:
# Water level data
# For a fixed station in Kotzebue, AK (https://www.google.com/maps?q=66.895035,-162.566752)
filename = basedir.joinpath('water_level_example.csv')
variable_name='sea_surface_height_above_sea_level'

# Load data
data = pd.read_csv(filename, parse_dates=['time'])

from ioos_qc.streams import PandasStream

ps = PandasStream(data)

In [ ]:
# Run QC
qc_results = ps.run(c)

In [ ]:
# Plot results
title = "Water Level [MHHW] [m] : Kotzebue, AK"

bokeh_multi_plot(ps, qc_results, title, ncols=2, sizing_mode='scale_width')